In [2]:
import pandas as pd
import plotly.graph_objects as go
from datetime import datetime
from zoneinfo import ZoneInfo
import re

# -------------------------
# LOAD DATA
# -------------------------
df = pd.read_csv("googleplaystore.csv")

# -------------------------
# DATA CLEANING
# -------------------------

# Clean Installs
df["Installs"] = (
    df["Installs"]
    .astype(str)
    .str.replace("+", "", regex=False)
    .str.replace(",", "", regex=False)
)
df["Installs"] = pd.to_numeric(df["Installs"], errors="coerce")

# Clean Price → Revenue calculation
df["Price"] = (
    df["Price"]
    .astype(str)
    .str.replace("$", "", regex=False)
)
df["Price"] = pd.to_numeric(df["Price"], errors="coerce")

# Revenue = Installs × Price
df["Revenue"] = df["Installs"] * df["Price"]

# Clean Rating
df["Rating"] = pd.to_numeric(df["Rating"], errors="coerce")

# Convert Size to MB
def size_to_mb(x):
    try:
        if isinstance(x, str):
            if "M" in x:
                return float(x.replace("M", ""))
            elif "k" in x:
                return float(x.replace("k", "")) / 1024
        return None
    except:
        return None

df["Size_MB"] = df["Size"].apply(size_to_mb)

# Extract Android Version (numeric)
def extract_android_version(x):
    try:
        match = re.search(r"\d+(\.\d+)?", str(x))
        return float(match.group()) if match else None
    except:
        return None

df["Android_Version"] = df["Android Ver"].apply(extract_android_version)

# -------------------------
# APPLY FILTERS
# -------------------------

filtered = df[
    (df["Installs"] >= 10000) &
    (df["Revenue"] >= 10000) &
    (df["Android_Version"] > 4.0) &
    (df["Size_MB"] > 15) &
    (df["Content Rating"] == "Everyone") &
    (df["App"].str.len() <= 30)
]

# -------------------------
# TOP 3 CATEGORIES BY INSTALLS
# -------------------------

top_categories = (
    filtered.groupby("Category")["Installs"]
    .sum()
    .sort_values(ascending=False)
    .head(3)
    .index
)

top_df = filtered[filtered["Category"].isin(top_categories)]

# -------------------------
# AGGREGATE FREE vs PAID
# -------------------------

summary = (
    top_df.groupby(["Category", "Type"])
    .agg(
        Avg_Installs=("Installs", "mean"),
        Avg_Revenue=("Revenue", "mean")
    )
    .reset_index()
)

# -------------------------
# TIME CONDITION (1PM–2PM IST)
# -------------------------

current_time = datetime.now(ZoneInfo("Asia/Kolkata"))
current_hour = current_time.hour

print("Current IST Time:", current_time.strftime("%Y-%m-%d %H:%M:%S"))

if 13 <= current_hour < 14 and not summary.empty:

    fig = go.Figure()

    # Average Installs (Primary Y Axis)
    fig.add_trace(go.Bar(
        x=summary["Category"] + " (" + summary["Type"] + ")",
        y=summary["Avg_Installs"],
        name="Average Installs",
        yaxis="y1"
    ))

    # Average Revenue (Secondary Y Axis)
    fig.add_trace(go.Scatter(
        x=summary["Category"] + " (" + summary["Type"] + ")",
        y=summary["Avg_Revenue"],
        name="Average Revenue ($)",
        yaxis="y2",
        mode="lines+markers"
    ))

    fig.update_layout(
        title="Top 3 Categories (Free vs Paid)\nVisible Only Between 1 PM IST and 2 PM IST",
        xaxis_title="Category (App Type)",
        yaxis=dict(title="Average Installs"),
        yaxis2=dict(
            title="Average Revenue ($)",
            overlaying="y",
            side="right"
        ),
        template="plotly_white"
    )

    fig.show()

else:
    print("⚠ Dashboard Locked")
    print("Graph is visible ONLY between 1 PM IST and 2 PM IST.")


Current IST Time: 2026-02-16 09:06:39
⚠ Dashboard Locked
Graph is visible ONLY between 1 PM IST and 2 PM IST.
